In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,82.36,75,51,15.37,light rain
1,1,Severo-Kurilsk,RU,50.6789,156.1250,31.41,79,100,11.81,overcast clouds
2,2,Lorengau,PG,-2.0226,147.2712,82.96,79,100,8.50,overcast clouds
3,3,Tiszaluc,HU,48.0333,21.0833,39.09,80,100,13.96,light rain
4,4,Kijang,KR,35.2442,129.2139,31.73,42,19,5.10,few clouds


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,82.36,75,51,15.37,light rain
2,2,Lorengau,PG,-2.0226,147.2712,82.96,79,100,8.50,overcast clouds
5,5,Sola,VU,-13.8833,167.5500,82.94,80,100,19.80,overcast clouds
6,6,Kailua,US,21.4022,-157.7394,84.18,72,21,8.01,few clouds
8,8,Georgetown,MY,5.4112,100.3354,76.95,91,40,2.30,scattered clouds
9,9,Ciudad Bolivar,VE,8.1222,-63.5497,84.13,59,86,15.75,overcast clouds
13,13,Hilo,US,19.7297,-155.0900,78.53,75,100,11.50,overcast clouds
17,17,Quatre Cocos,MU,-20.2078,57.7625,78.91,84,94,9.10,light rain
24,24,Noumea,NC,-22.2763,166.4572,84.18,81,56,9.71,broken clouds
30,30,Atambua,ID,-9.1061,124.8925,79.59,77,42,2.04,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                173
City                   173
Country                173
Lat                    173
Lng                    173
Max Temp               173
Humidity               173
Cloudiness             173
Wind Speed             173
Current Description    173
dtype: int64

In [9]:
#Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,82.36,75,51,15.37,light rain
2,2,Lorengau,PG,-2.0226,147.2712,82.96,79,100,8.50,overcast clouds
5,5,Sola,VU,-13.8833,167.5500,82.94,80,100,19.80,overcast clouds
6,6,Kailua,US,21.4022,-157.7394,84.18,72,21,8.01,few clouds
8,8,Georgetown,MY,5.4112,100.3354,76.95,91,40,2.30,scattered clouds
9,9,Ciudad Bolivar,VE,8.1222,-63.5497,84.13,59,86,15.75,overcast clouds
13,13,Hilo,US,19.7297,-155.0900,78.53,75,100,11.50,overcast clouds
17,17,Quatre Cocos,MU,-20.2078,57.7625,78.91,84,94,9.10,light rain
24,24,Noumea,NC,-22.2763,166.4572,84.18,81,56,9.71,broken clouds
30,30,Atambua,ID,-9.1061,124.8925,79.59,77,42,2.04,scattered clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,82.36,light rain,3.0707,172.7902,
2,Lorengau,PG,82.96,overcast clouds,-2.0226,147.2712,
5,Sola,VU,82.94,overcast clouds,-13.8833,167.5500,
6,Kailua,US,84.18,few clouds,21.4022,-157.7394,
8,Georgetown,MY,76.95,scattered clouds,5.4112,100.3354,
9,Ciudad Bolivar,VE,84.13,overcast clouds,8.1222,-63.5497,
13,Hilo,US,78.53,overcast clouds,19.7297,-155.0900,
17,Quatre Cocos,MU,78.91,light rain,-20.2078,57.7625,
24,Noumea,NC,84.18,broken clouds,-22.2763,166.4572,
30,Atambua,ID,79.59,scattered clouds,-9.1061,124.8925,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,82.36,light rain,3.0707,172.7902,Isles Sunset Lodge
2,Lorengau,PG,82.96,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
5,Sola,VU,82.94,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
6,Kailua,US,84.18,few clouds,21.4022,-157.7394,Sheffield House Bed & Breakfast
8,Georgetown,MY,76.95,scattered clouds,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...,...
654,Bowen,AU,80.82,broken clouds,-20.0167,148.2333,Castle Motor Lodge Motel
659,Puerto Rondon,CO,80.65,scattered clouds,6.2805,-71.1000,Urbanización Santa Clara
665,Mildura,AU,83.07,clear sky,-34.2000,142.1500,Mercure Hotel Mildura
666,Takaungu,KE,75.81,overcast clouds,-3.6837,39.8566,Takaungu House


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
clean_hotel_df = hotel_df

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))